# ETL Google Business (Extract, Load, Transform)

This project aims to explore and analyze the valuable insights and opinions expressed by users in business reviews on Google. In this context, the Extract, Transform, Load (ETL) process takes on central importance. We will be working with data sourced from businesses registered on Google Maps (specifically focusing on Ulta Beauty), utilizing ETL techniques to ensure the efficient collection, transformation, and preparation of the data.

### Requirements

⚠️ **Make sure to install the following libraries before running the code**

- pandas
- numpy

You can install these libraries by opening a terminal or command line window and running the following command:

*`pip install pandas numpy`*

## 1. Import Libraries

In [1]:
import pandas as pd
import os

## 2. Connect and Upload Data

In [2]:
# Read the files and process the json contained in each file one by one

jsons=os.listdir(r"D:\Datasets_proyecto\metadata-sitios")

for a in jsons:
    ruta=f'D:\Datasets_proyecto\metadata-sitios\\{a}'
    dfbusinessGoogle=pd.read_json(ruta,lines=True)

## 3. Explore and Clean Data

In [3]:
# A filter is applied to the entire data set, since the scope of the project is oriented to the "Ulta Beauty" business.

dfbusinessGoogle = dfbusinessGoogle[dfbusinessGoogle['name'] == 'Ulta Beauty']

In [5]:
# Businesses that are permanently closed are filtered

dfbusinessGoogle = dfbusinessGoogle[dfbusinessGoogle['state'] != 'Permanently closed']

In [6]:
# From the 'address' column, we extracted the state, city, and corresponding postal code. 

dfbusinessGoogle['address'] = dfbusinessGoogle['address'].str.replace('Ulta Beauty,', '').str.strip()

dfbusinessGoogle['state']=dfbusinessGoogle['state_PostalCode'].str.split(' ').str[1]
dfbusinessGoogle['city'] = dfbusinessGoogle['address'].str.split(',').str[1]
dfbusinessGoogle['state_PostalCode'] = dfbusinessGoogle['address'].str.split(',').str[2]
dfbusinessGoogle['postal_code']=dfbusinessGoogle['state_PostalCode'].str.split(' ').str[2]

**The following columns are eliminated because they are not relevant to the project**

In [ ]:
#name
dfbusinessGoogle = dfbusinessGoogle.drop('name', axis=1) #All the stores have the same name "Ulta Beauty"

#state
dfbusinessGoogle = dfbusinessGoogle.drop('state', axis=1) 

#address
dfbusinessGoogle = dfbusinessGoogle.drop('address', axis=1)

#state_PostalCode
dfbusinessGoogle = dfbusinessGoogle.drop('state_PostalCode', axis=1)

#description
dfbusinessGoogle = dfbusinessGoogle.drop('description', axis=1)

#category
dfbusinessGoogle = dfbusinessGoogle.drop('category', axis=1)

#price
dfbusinessGoogle = dfbusinessGoogle.drop('price', axis=1) #All it´s data is null

#relative_results
dfbusinessGoogle = dfbusinessGoogle.drop('relative_results', axis=1)

#url
dfbusinessGoogle = dfbusinessGoogle.drop('url', axis=1)

#MISC
dfbusinessGoogle = dfbusinessGoogle.drop('MISC', axis=1)

#hours
dfbusinessGoogle = dfbusinessGoogle.drop('hours', axis=1)

**Handling null and duplicate values**

In [13]:
dfbusinessGoogle.isnull().sum()

gmap_id           0
latitude          0
longitude         0
avg_rating        0
num_of_reviews    0
city              0
state             0
postal_code       0
dtype: int64

No null values were found in the dataset

In [14]:
# Get the number of rows before removing duplicates
num_filas_antes = dfbusinessGoogle.shape[0]

# Remove duplicates based on 'address' column
dfbusinessGoogle = dfbusinessGoogle.drop_duplicates()

# Get the number of rows after removing duplicates
num_filas_despues = dfbusinessGoogle.shape[0]

# Show the number of duplicates before and after
print(f"Number of duplicates deleted: {num_filas_antes - num_filas_despues}")


Cantidad de duplicados borrados: 0


**Columns and Rows Normalization**

In [15]:
#The columns will be named consistently with those in the "Yelp" dataset

dfbusinessGoogle = dfbusinessGoogle.rename(columns={'gmap_id': 'business_id', 'avg_rating': 'stars', 'num_of_reviews': 'review_count'})

In [16]:
# The names of the cities will be converted to lowercase and the spaces will be replaced by underscores

dfbusinessGoogle['city'] = dfbusinessGoogle['city'].str.strip().str.lower().str.replace(' ', '_')

In [17]:
# The acronyms are replaced by the names of the states

estado_mapping = {'TX': 'texas', 'MT': 'montana', 'CA': 'california', 'FL': 'florida',
                  'IL': 'illinois', 'MD': 'maryland', 'MA': 'massachusetts', 'PA': 'pennsylvania',
                  'NY': 'new_york', 'OH': 'ohio', 'OK': 'oklahoma', 'MO': 'missouri',
                  'CT': 'connecticut', 'NJ': 'new_jersey', 'NC': 'north_carolina'}


# Replace acronyms with names in the 'state_postalCode' column

dfbusinessGoogle['state'] = dfbusinessGoogle['state'].replace(estado_mapping)

In [22]:
# We adjusted the column order to match the structure of the Yelp dataset.

nuevo_orden_columnas = ['business_id', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count']

dfbusinessGoogle = dfbusinessGoogle[nuevo_orden_columnas]

In [24]:
# The 'source' column is added as an identifier.
# G = Data that comes from the google dataset

dfbusinessGoogle['source']="G"

## 4. Final Structure

In [25]:
dfbusinessGoogle.head()

,business_id,city,state,postal_code,latitude,longitude,stars,review_count,source
3858,0x864c394f020aaa49:0x20e59a02a019a45d,frisco,texas,75034,33.173242,-96.843504,3.9,143,G
37741,0x5342381c6ddf30e3:0xfbe922695b89d6de,great_falls,montana,59404,47.518201,-111.315605,4.1,164,G
53920,0x808fe985ad5d01df:0x20e8c372f20b006c,livermore,california,94551,37.696312,-121.847587,4.3,66,G
72836,0x88d91958f9f6f4ab:0x111e84f49744687d,boca_raton,florida,33434,26.364980,-80.202167,4.0,158,G
82511,0x880fa30ab6b747f3:0x2a88bf8178a2c164,kildeer,illinois,60047,42.162774,-88.050782,3.8,8,G
